In [1]:
#MPNN_cust_feat
import deepchem as dc
import os
import sys
os.environ['DGLBACKEND'] = 'pytorch'
import numpy as np
import pandas as pd
import dgl
import torch
from deepchem.feat.graph_data import GraphData
from deepchem.models import MPNNModel
from mpnn_featurizer import MPNNFeaturizer
from mpnn_normalizer import MPNNNormalizer

import math
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:32'
import copy
sys.path.append('../')
from utils import *
from save_data import *
from run_experiments import run_experiments

from itertools import product
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

2024-08-06 17:04:53.231072: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 17:04:53.233862: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2024-08-06 17:04:53.233873: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [22]:
#delete me
scaffolds=['O=c1cc[nH]c(=O)[nH]1']
df = pd.read_csv('../expt_phys_data/full_dataset_with_rigid.csv')
df[(df.scaffold == scaffolds[0])]

,id,smiles,iupac name,scaffold,expt,uncertainty,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,scaffold_split,uncertain_expt_value,rigid,only_HCNO
78,mobley_194273,CC[C@H](C)n1c(=O)c(c([nH]c1=O)C)Br,bromacil,O=c1cc[nH]c(=O)[nH]1,-9.73,1.93,-14.496,-14.4769,-10.21470,-12.2162,-13.664,-14.286,0,True,0,2,0,1,0.5,False
149,mobley_2727678,c1c(c(=O)[nH]c(=O)[nH]1)I,5-iodouracil,O=c1cc[nH]c(=O)[nH]1,-18.72,0.64,-17.742,-18.2348,-14.13590,-17.6760,-20.982,-20.654,0,True,0,2,0,0,0.5,False
185,mobley_3105103,Cc1c(c(=O)n(c(=O)[nH]1)C(C)(C)C)Cl,terbacil,O=c1cc[nH]c(=O)[nH]1,-11.14,1.93,-13.769,-13.9969,-9.59913,-10.7931,-12.391,-12.292,0,True,0,2,0,1,0.5,False
209,mobley_337666,c1c(c(=O)[nH]c(=O)[nH]1)F,5-fluorouracil,O=c1cc[nH]c(=O)[nH]1,-16.92,0.88,-16.371,-17.5375,-14.20340,-17.3403,-21.404,-20.520,0,True,0,2,0,0,0.5,False
252,mobley_4039055,c1c(c(=O)[nH]c(=O)[nH]1)Br,5-bromouracil,O=c1cc[nH]c(=O)[nH]1,-18.17,0.55,-17.298,-18.5222,-14.69900,-16.9260,-20.386,-19.947,0,False,2,2,0,0,0.5,False
267,mobley_4338603,c1c(=O)[nH]c(=O)[nH]c1Cl,6-chlorouracil,O=c1cc[nH]c(=O)[nH]1,-15.83,1.22,-15.128,-16.0786,-12.72740,-14.7955,-17.948,-17.840,0,True,1,2,0,0,0.5,False
509,mobley_7794077,c1c(c(=O)[nH]c(=O)[nH]1)C(F)(F)F,5-trifluoromethyluracil,O=c1cc[nH]c(=O)[nH]1,-15.46,0.16,-17.349,-16.8455,-14.47460,-18.1672,-22.520,-22.705,0,True,0,2,0,0,0.5,False
617,mobley_9557440,c1c(c(=O)[nH]c(=O)[nH]1)Cl,5-chlorouracil,O=c1cc[nH]c(=O)[nH]1,-17.74,0.78,-16.612,-17.9397,-14.82650,-16.6514,-20.390,-19.853,0,True,0,2,0,0,0.5,False


In [2]:
df = pd.read_csv('../expt_phys_data/full_dataset_with_rigid.csv')

# remove molecules that give error with featurizer
df = df[(df.smiles != 'C') & (df.smiles != 'S') & (df.smiles != 'N')]
# df = df[df.uncertain_expt_value == 0]
split = 'scaffold_split'#hilo_split,paper_split,'scaffold_split'
df_train = df[(df[split] == 0)]
df_valid = df[(df[split] == 1)]
df_test = df[(df[split] == 2)]

# df
df_amino = pd.read_csv('../expt_phys_data/amino_dataset.csv')
print(len(df_train),len(df_valid),len(df_test))

#set used for training models always is first. if early stopping is used,
#second set is used for that
# dataset_names = ['train','valid','test','amino']
dataset_names = ['train','valid','test']
norm_datasets = ['train','valid']
# dfs = [df_train,df_valid,df_test,df_amino]
dfs = [df_train,df_valid,df_test]

479 80 80


In [23]:
df

,id,smiles,iupac name,scaffold,expt,uncertainty,tip3p,cha,zap9,mbondi,asc,igb5,null,train,paper_split,hilo_split,scaffold_split,uncertain_expt_value,rigid,only_HCNO
0,mobley_1017962,CCCCCC(=O)OC,methyl hexanoate,NaN,-2.49,0.6,-3.299,-3.726220,-3.004340,-4.006280,-3.886,-5.059,0,False,2,1,0,0,0.0,True
1,mobley_1019269,CCCCO,butan-1-ol,NaN,-4.72,0.6,-3.232,-3.135990,-3.222270,-5.188390,-3.080,-4.198,0,False,2,0,1,0,0.0,True
2,mobley_1034539,c1cc(c(cc1c2cc(c(c(c2Cl)Cl)Cl)Cl)Cl)Cl,"1,2,3,4-tetrachloro-5-(3,4-dichlorophenyl)benzene",c1ccc(-c2ccccc2)cc1,-3.04,0.1,-1.083,-2.344750,-2.113720,-0.459959,0.899,-1.348,0,False,2,0,1,0,0.5,False
3,mobley_1036761,C1CCC(CC1)N,cyclohexanamine,C1CCCCC1,-4.59,0.6,-3.953,-1.755250,-0.447715,-0.027103,0.240,-2.215,0,True,1,0,2,0,1.0,True
4,mobley_1046331,c1ccc(cc1)OC=O,phenyl formate,c1ccccc1,-3.82,0.6,-5.442,-5.706530,-6.577580,-7.407990,-8.342,-10.084,0,True,0,0,0,0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,mobley_9913368,C(=C/Cl)\Cl,"(E)-1,2-dichloroethylene",NaN,-0.78,0.6,1.024,-0.252349,-0.595880,-0.520564,-0.090,-1.126,0,True,1,0,0,0,1.0,False
638,mobley_9942801,CCc1ccc(cc1)C,1-ethyl-4-methyl-benzene,c1ccccc1,-0.95,0.6,-0.575,-1.201310,-0.421314,-0.758853,0.059,-1.641,0,True,0,0,0,0,0.0,True
639,mobley_994483,CCBr,bromoethane,NaN,-0.74,0.6,0.487,-0.821768,-0.062573,-1.089090,-0.095,-0.818,0,True,0,0,0,0,1.0,False
640,mobley_9974966,CC(C)SC(C)C,2-isopropylsulfanylpropane,NaN,-1.21,0.6,0.140,-1.111810,0.499588,-0.765837,0.032,-1.358,0,True,0,0,0,0,0.0,False


In [3]:
# the following should only be used when hydrogens are implicit:
# implicit_valence, num_attached_H (both this and last are almost identical)
# to add in future: sum_attached_H_gb, sum_attached_H_charge
# note: explicit_valence changes based on whether H are implicit
# node: normalization level is listed after feature name:
#          0: none, 1: min-max, 2: z-score      
#       only normalize real valued entries (not binary or one-hot)
feat_params_dict = {
    'feats':[
        # {
        #     'node_feats':['atom_identity','partial_charge','atom_gb','sum_pairwise_gb',
        #                   'inverse_born_radius','mbondi_radius','int_charge','hybridization',
        #                  'hydrogen_bonding','aromatic','degree','num_attached_H','implicit_valence',
        #                  'explicit_valence'],
        #     'edge_feats':['distance','pairwise_gb','bonded','bond_type','same_ring',
        #                  'conjugated','stereo_config'],
        # },
        # {
        #     'node_feats':{'atom_identity':0,'int_charge':0,'hybridization':0,'hydrogen_bonding':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'bond_type':0,'same_ring':0,'conjugated':0,'stereo_config':0},
        # },
        # {
        #     'node_feats':{'partial_charge':2,'atom_gb':1,'sum_pairwise_gb':2,'electronegativity':1,
        #                   'flexibility':0,'inverse_born_radius':1,
        #                   'atom_identity':0,'int_charge':0,'hybridization':0,'hydrogen_bonding':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':1,'pairwise_gb':2,'bond_type':0,'same_ring':0,'conjugated':0,'stereo_config':0},
        # },
        {
            'node_feats':{
                          'atom_identity':0,'hybridization':0,
                          'aromatic':0,'degree':0,'num_attached_H':0},
            'edge_feats':{'bond_type':0},
        },
        {
            'node_feats':{'partial_charge':1,'inverse_born_radius':1,
                          },
            'edge_feats':{'distance':1},
        },
        {
            'node_feats':{'partial_charge':1,'inverse_born_radius':1,
                          'atom_identity':0,'hybridization':0,
                          'aromatic':0,'degree':0,'num_attached_H':0},
            'edge_feats':{'distance':1,'bond_type':0},
        },
        # {
        #     'node_feats':{
        #                   'atom_identity':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'bond_type':0},
        # },
        # {
        #     'node_feats':{'partial_charge':1,
        #                   'atom_identity':0,'hybridization':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':1,'bond_type':0},
        # },
        # {
        #     'node_feats':{'partial_charge':1,
        #                   'atom_identity':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':1,'bond_type':0},
        # },
        # {
        #     'node_feats':{'partial_charge':1,'inverse_born_radius':1,
        #                   'atom_identity':0,'int_charge':0,'hybridization':0,'hydrogen_bonding':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':1,'bond_type':0,'same_ring':0,'conjugated':0,'stereo_config':0},
        # },
        # {
        #     'node_feats':{'partial_charge':2,'inverse_born_radius':2,
        #                   'atom_identity':0,'int_charge':0,'hybridization':0,'hydrogen_bonding':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':2,'bond_type':0,'same_ring':0,'conjugated':0,'stereo_config':0},
        # },
        # {
        #     'node_feats':{'partial_charge':2,'inverse_born_radius':1,
        #                   'atom_identity':0,'int_charge':0,'hybridization':0,'hydrogen_bonding':0,
        #                   'aromatic':0,'degree':0,'num_attached_H':0},
        #     'edge_feats':{'distance':1,'bond_type':0,'same_ring':0,'conjugated':0,'stereo_config':0},
        # },
        # {
        #     'node_feats':{'partial_charge':2,'inverse_born_radius':1},
        #     'edge_feats':{'distance':1},
        # },
        # {
        #     'node_feats':[],
        #     'edge_feats':[],
        # },
    ],
    'edges':[
        # {
        #     'edge_cutoff_variable':'fully_connected',#fully_connected,pairwise_gb, distance, bonds
        # },
        {
            'edge_cutoff_variable':'bonds',#pairwise_gb, distance, bonds
            'implicit_h':True
        },
        # {
        #     'edge_cutoff_variable':'bonds',#pairwise_gb, distance, bonds
        #     'implicit_h':False
        # },
        # {
        #     'edge_cutoff_variable':'fully_connected',#pairwise_gb, distance, bonds
        #     'implicit_h':True
        # },
        # {
        #     'edge_cutoff_variable':'fully_connected',#pairwise_gb, distance, bonds
        #     'implicit_h':False
        # },
    ],
    
    
}
#comment out whatever parameter to use default
hyper_params_dict = {
    'epochs':[200],
    # 'batch_size':[500,100,50,20],#100
    # 'learning_rate':[0.001,0.0005,0.0001], #0.001
    # 'node_out_feats':[32], #64
    # 'edge_hidden_feats':[256], #128
    # 'num_step_message_passing':[2], #3
    # 'num_step_set2set':[], #6
    # 'num_layer_set2set':[], #3
    #comment out following for no early stopping
    'es_overfitting':[1.0],
    'es_over_patience':[20],
    'es_min_delta':[0.01],
    'es_conv_patience':[20],
    'iter':[20],
}

feat_param_combos = list(product(*feat_params_dict.values()))
hyper_combos = list(product(*hyper_params_dict.values()))

In [4]:
# folder_base = 'prelim2_scaffold_test_results/'
folder_base = 'final_results_full_data_scaffold_split/'    
# feat_param_combos,hyper_combos,feat_params_dict,hyper_params_dict = get_param_combos(folder_base)
# hyper_combos = [hyper_combos[i] for i in[0,3]]
# feat_param_combos = [feat_param_combos[i] for i in[0,3]]

print(f'num feat combos = {len(feat_param_combos)}, num hyper combos = {len(hyper_combos)}, total combos = {len(feat_param_combos)*len(hyper_combos)}')
#if you do not have hyper parameter and feature parameters in json files in folder_base
#then you should define them in the cell above and uncomment the below code

# if not os.path.isdir(folder_base):
#     os.makedirs(folder_base)
save_params(feat_params_dict,folder_base,'feat_params')
save_params(hyper_params_dict,folder_base,'hyper_params')



num feat combos = 3, num hyper combos = 1, total combos = 3


In [5]:
folder_list = []
featurize = True #true if you want to refeaturize instead of loading datasets
                  #set true if you want MPNNMolecule objects which include additional info
delete_checkpoints = False #true is you want checkpoints deleted
normalize_targets = 0
hyper_folder_prefix = ''
model_type = 'mpnn' #graphconv for paper model

phys_models = ['tip3p','cha','zap9','mbondi','asc','igb5','null']
# phys_models = ['null','zap9','mbondi','tip3p']
# phys_models = ['null','tip3p']

iter = 2 #number of duplicate models to train in each specific case

In [ ]:
%%capture output --no-stderr
%%time
# %%capture output --no-stderr

for params in feat_param_combos:
    feat_params = dict(zip(feat_params_dict.keys(), params))

    feat_folder = create_folder_name(folder_base,feat_params,'custom_mpnn')

    if((os.path.exists(feat_folder+'datasets.pkl')) and (not featurize)):
        Xs,dfs,dataset_names,featurizer,normalizer = load_saved_datasets(feat_folder+'datasets.pkl')
    else:
        featurizer = MPNNFeaturizer(feat_params)
    
        Xs = []
        molecule_sets = []
        norm_X = []
        for i in range(len(dfs)):
            X,molecules = featurizer.featurize(dfs[i],dataset_names[i])
            Xs.append(X)
            molecule_sets.append(molecules)
            if(dataset_names[i] in norm_datasets):
                norm_X += X
        #normalize here if necessary (always based on entire train set)
        normalizer = MPNNNormalizer(feat_params,molecule_sets[0][0])
        normalizer.fit(norm_X)
        for X in Xs:
           normalizer.transform(X)
    
        save_datasets(feat_folder,Xs,dfs,dataset_names,featurizer)
    
    for hyper in hyper_combos:
        hyper_params = dict(zip(hyper_params_dict.keys(), hyper))
        num_iter = hyper_params['iter']
        hyper_folder = create_folder_name(feat_folder+hyper_folder_prefix,hyper_params)
        
        print(hyper_folder)

        #trains residual models for each physics model in 'phys_models'
        #number of models per is 'iter'. saves all results in 'hyper_folder'
        #print_results: 0 for nothing, 1 for summary data, 2 for data of all models
        run_experiments(Xs, dfs, phys_models,model_type,hyper_params,num_iter,dataset_names,\
                        hyper_folder,folder_list,normalize_targets,norm_datasets,\
                        device=torch.device('cuda'),print_results = 1,with_target_noise=False)
        
        #for cross validation (or any validation) train set is given to 
        #    validation function which splits it and calls experiment
        torch.cuda.empty_cache()
    save_params(feat_params,feat_folder)
save_params({'folder_list':folder_list},folder_base,'folder_list')

ens_dfs,mean_dfs,epoch_df,dataset_names = generate_summary_dfs(folder_list,phys_models)
save_summary_dfs(ens_dfs,mean_dfs,epoch_df,dataset_names,folder_base)

if(delete_checkpoints):
    delete_all_checkpoints(folder_list)

In [ ]:
with open(folder_base+'output.txt', 'w') as f:
    f.write(str(output))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
#filter uncertain:
# prelim_final_results/feats-node_z-hy-ar-de-nh-edge_bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20-iter_3/
# TIP3P
# Total models: 3, mean number of epochs: 51.7 ± 12.5
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.477 ± 0.183  | 0.764 ± 0.074  | 0.860 ± 0.034  
# ensemble ml   | 0.399          | 0.692          | 0.785          

# CHA-GB
# Total models: 3, mean number of epochs: 45.3 ± 11.6
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.690 ± 0.136  | 1.066 ± 0.100  | 0.931 ± 0.018  
# ensemble ml   | 0.585          | 0.965          | 0.752          

# ZAP9
# Total models: 3, mean number of epochs: 37.7 ± 11.0
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.697 ± 0.177  | 1.086 ± 0.153  | 0.996 ± 0.072  
# ensemble ml   | 0.606          | 1.012          | 0.880          

# mbondi
# Total models: 3, mean number of epochs: 37.3 ± 11.8
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.846 ± 0.165  | 1.292 ± 0.048  | 1.051 ± 0.102  
# ensemble ml   | 0.761          | 1.206          | 0.955          

# AASC
# Total models: 3, mean number of epochs: 46.7 ± 23.1
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.837 ± 0.256  | 1.199 ± 0.135  | 1.170 ± 0.239  
# ensemble ml   | 0.717          | 1.085          | 1.004          

# IGB5
# Total models: 3, mean number of epochs: 73.7 ± 23.5
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.842 ± 0.277  | 1.330 ± 0.099  | 1.302 ± 0.241  
# ensemble ml   | 0.728          | 1.161          | 1.181          

# ML alone
# Total models: 3, mean number of epochs: 76.7 ± 16.5
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.652 ± 0.029  | 1.215 ± 0.049  | 1.244 ± 0.048  
# ensemble ml   | 0.529          | 1.055          | 1.124


# prelim_final_results/feats-node_c-br-edge_r-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20-iter_3/
# TIP3P
# Total models: 3, mean number of epochs: 65.7 ± 26.3
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.921 ± 0.110  | 1.008 ± 0.061  | 1.000 ± 0.028  
# ensemble ml   | 0.872          | 0.953          | 0.959          

# CHA-GB
# Total models: 3, mean number of epochs: 73.0 ± 23.8
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 1.064 ± 0.114  | 1.414 ± 0.087  | 1.122 ± 0.085  
# ensemble ml   | 0.982          | 1.337          | 1.014          

# ZAP9
# Total models: 3, mean number of epochs: 85.0 ± 21.2
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.890 ± 0.101  | 1.301 ± 0.061  | 1.107 ± 0.020  
# ensemble ml   | 0.817          | 1.253          | 1.027          

# mbondi
# Total models: 3, mean number of epochs: 90.0 ± 14.1
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.919 ± 0.030  | 1.271 ± 0.012  | 1.134 ± 0.070  
# ensemble ml   | 0.834          | 1.198          | 1.055          

# AASC
# Total models: 3, mean number of epochs: 73.7 ± 19.9
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 1.120 ± 0.219  | 1.515 ± 0.235  | 1.340 ± 0.086  
# ensemble ml   | 0.925          | 1.331          | 1.084          

# IGB5
# Total models: 3, mean number of epochs: 76.3 ± 17.3
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.913 ± 0.143  | 1.312 ± 0.159  | 1.229 ± 0.055  
# ensemble ml   | 0.837          | 1.194          | 1.159          

# ML alone
# Total models: 3, mean number of epochs: 100.0 ± 0.0
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 1.962 ± 0.388  | 2.491 ± 0.232  | 2.324 ± 0.240  
# ensemble ml   | 1.625          | 2.023          | 1.753          

# prelim_final_results/feats-node_c-br-z-hy-ar-de-nh-edge_r-bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20-iter_3/
# TIP3P
# Total models: 3, mean number of epochs: 56.0 ± 33.1
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.457 ± 0.254  | 0.796 ± 0.073  | 0.839 ± 0.070  
# ensemble ml   | 0.385          | 0.710          | 0.754          

# CHA-GB
# Total models: 3, mean number of epochs: 35.7 ± 8.3
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.809 ± 0.173  | 1.163 ± 0.089  | 0.924 ± 0.070  
# ensemble ml   | 0.703          | 1.089          | 0.795          

# ZAP9
# Total models: 3, mean number of epochs: 64.7 ± 25.1
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.405 ± 0.102  | 0.984 ± 0.036  | 0.945 ± 0.109  
# ensemble ml   | 0.350          | 0.920          | 0.896          

# mbondi
# Total models: 3, mean number of epochs: 29.0 ± 3.7
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.817 ± 0.104  | 1.326 ± 0.025  | 1.109 ± 0.053  
# ensemble ml   | 0.761          | 1.273          | 1.065          

# AASC
# Total models: 3, mean number of epochs: 45.3 ± 13.9
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.755 ± 0.173  | 1.186 ± 0.074  | 1.050 ± 0.092  
# ensemble ml   | 0.674          | 1.098          | 0.917          

# IGB5
# Total models: 3, mean number of epochs: 49.3 ± 7.4
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.853 ± 0.081  | 1.275 ± 0.074  | 1.170 ± 0.117  
# ensemble ml   | 0.773          | 1.205          | 1.077          

# ML alone
# Total models: 3, mean number of epochs: 57.7 ± 6.6
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.862 ± 0.070  | 1.576 ± 0.172  | 1.250 ± 0.098  
# ensemble ml   | 0.731          | 1.378          | 1.100          

# CPU times: user 40min 12s, sys: 35.7 s, total: 40min 48s
# Wall time: 15min 53s

In [ ]:
#compare with and without hybridization
# delete_me/feats-node_z-hy-ar-de-nh-edge_bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 56.2 ± 20.6
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.520 ± 0.147  | 0.799 ± 0.076  | 0.843 ± 0.055  
# ensemble ml   | 0.428          | 0.689          | 0.757          

# CHA-GB
# Total models: 10, mean number of epochs: 47.7 ± 8.1
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.644 ± 0.144  | 1.078 ± 0.108  | 0.903 ± 0.069  
# ensemble ml   | 0.523          | 0.946          | 0.742          

# ZAP9
# Total models: 10, mean number of epochs: 57.3 ± 20.2
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.505 ± 0.113  | 1.013 ± 0.097  | 0.985 ± 0.077  
# ensemble ml   | 0.391          | 0.895          | 0.874          

# mbondi
# Total models: 10, mean number of epochs: 37.4 ± 11.3
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.843 ± 0.170  | 1.332 ± 0.098  | 1.109 ± 0.136  
# ensemble ml   | 0.731          | 1.213          | 1.005          

# AASC
# Total models: 10, mean number of epochs: 46.9 ± 15.4
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.797 ± 0.194  | 1.319 ± 0.104  | 1.140 ± 0.136  
# ensemble ml   | 0.656          | 1.139          | 0.972          

# IGB5
# Total models: 10, mean number of epochs: 55.2 ± 10.7
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.864 ± 0.105  | 1.337 ± 0.051  | 1.245 ± 0.087  
# ensemble ml   | 0.746          | 1.213          | 1.112          

# ML alone
# Total models: 10, mean number of epochs: 68.7 ± 17.6
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.722 ± 0.170  | 1.334 ± 0.168  | 1.219 ± 0.047  
# ensemble ml   | 0.578          | 1.114          | 1.059          

# delete_me/feats-node_z-ar-de-nh-edge_bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 40.6 ± 11.9
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.630 ± 0.109  | 0.837 ± 0.078  | 0.942 ± 0.070  
# ensemble ml   | 0.543          | 0.734          | 0.855          

# CHA-GB
# Total models: 10, mean number of epochs: 44.0 ± 13.7
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.701 ± 0.190  | 1.079 ± 0.103  | 0.906 ± 0.121  
# ensemble ml   | 0.567          | 0.943          | 0.719          

# ZAP9
# Total models: 10, mean number of epochs: 56.0 ± 13.2
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.469 ± 0.107  | 1.014 ± 0.045  | 0.972 ± 0.070  
# ensemble ml   | 0.374          | 0.911          | 0.862          

# mbondi
# Total models: 10, mean number of epochs: 32.7 ± 9.5
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.865 ± 0.133  | 1.296 ± 0.054  | 1.138 ± 0.067  
# ensemble ml   | 0.756          | 1.184          | 1.060          

# AASC
# Total models: 10, mean number of epochs: 47.8 ± 21.2
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.861 ± 0.172  | 1.250 ± 0.086  | 1.168 ± 0.087  
# ensemble ml   | 0.746          | 1.136          | 1.041          

# IGB5
# Total models: 10, mean number of epochs: 61.6 ± 21.0
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.832 ± 0.181  | 1.291 ± 0.071  | 1.234 ± 0.147  
# ensemble ml   | 0.707          | 1.150          | 1.067          

# ML alone
# Total models: 10, mean number of epochs: 69.8 ± 16.7
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.686 ± 0.188  | 1.252 ± 0.096  | 1.195 ± 0.115  
# ensemble ml   | 0.537          | 1.040          | 1.023          

# delete_me/feats-node_c-z-hy-ar-de-nh-edge_r-bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 40.5 ± 14.4
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.611 ± 0.158  | 0.852 ± 0.071  | 0.891 ± 0.070  
# ensemble ml   | 0.523          | 0.746          | 0.785          

# CHA-GB
# Total models: 10, mean number of epochs: 49.8 ± 11.3
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.626 ± 0.123  | 1.076 ± 0.097  | 0.915 ± 0.084  
# ensemble ml   | 0.491          | 0.928          | 0.773          

# ZAP9
# Total models: 10, mean number of epochs: 60.5 ± 15.5
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.459 ± 0.166  | 1.012 ± 0.078  | 0.981 ± 0.088  
# ensemble ml   | 0.351          | 0.888          | 0.837          

# mbondi
# Total models: 10, mean number of epochs: 35.8 ± 11.6
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.847 ± 0.148  | 1.293 ± 0.055  | 1.094 ± 0.102  
# ensemble ml   | 0.727          | 1.169          | 0.983          

# AASC
# Total models: 10, mean number of epochs: 41.1 ± 14.7
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.899 ± 0.157  | 1.292 ± 0.107  | 1.156 ± 0.112  
# ensemble ml   | 0.804          | 1.185          | 1.058          

# IGB5
# Total models: 10, mean number of epochs: 72.1 ± 20.9
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.774 ± 0.153  | 1.313 ± 0.075  | 1.280 ± 0.072  
# ensemble ml   | 0.646          | 1.188          | 1.143          

# ML alone
# Total models: 10, mean number of epochs: 66.9 ± 18.8
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.797 ± 0.221  | 1.389 ± 0.297  | 1.231 ± 0.142  
# ensemble ml   | 0.612          | 1.119          | 1.019          

# delete_me/feats-node_c-z-ar-de-nh-edge_r-bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 41.2 ± 9.2
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.582 ± 0.088  | 0.791 ± 0.048  | 0.923 ± 0.102  
# ensemble ml   | 0.494          | 0.691          | 0.795          

# CHA-GB
# Total models: 10, mean number of epochs: 44.4 ± 17.8
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.708 ± 0.223  | 1.113 ± 0.111  | 0.935 ± 0.091  
# ensemble ml   | 0.581          | 0.975          | 0.766          

# ZAP9
# Total models: 10, mean number of epochs: 60.0 ± 10.2
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.461 ± 0.086  | 0.998 ± 0.059  | 0.944 ± 0.074  
# ensemble ml   | 0.369          | 0.898          | 0.821          

# mbondi
# Total models: 10, mean number of epochs: 36.3 ± 9.9
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.825 ± 0.108  | 1.338 ± 0.085  | 1.136 ± 0.053  
# ensemble ml   | 0.737          | 1.248          | 1.037          

# AASC
# Total models: 10, mean number of epochs: 42.4 ± 5.9
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.869 ± 0.100  | 1.244 ± 0.065  | 1.125 ± 0.124  
# ensemble ml   | 0.786          | 1.116          | 1.022          

# IGB5
# Total models: 10, mean number of epochs: 74.3 ± 19.1
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.624 ± 0.131  | 1.244 ± 0.035  | 1.198 ± 0.105  
# ensemble ml   | 0.523          | 1.091          | 1.034          

# ML alone
# Total models: 10, mean number of epochs: 67.2 ± 7.2
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.749 ± 0.111  | 1.263 ± 0.199  | 1.233 ± 0.091  
# ensemble ml   | 0.588          | 1.043          | 1.075          

# CPU times: user 2h 41min 28s, sys: 2min 14s, total: 2h 43min 42s
# Wall time: 1h 4min 55s

In [ ]:
#final feature sets??
# delete_me/feats-node_z-hy-ar-de-nh-edge_bt-sr-co-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 56.9 ± 12.6
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.449 ± 0.109  | 0.753 ± 0.055  | 0.876 ± 0.068  
# ensemble ml   | 0.370          | 0.628          | 0.764          

# CHA-GB
# Total models: 10, mean number of epochs: 55.4 ± 18.8
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.515 ± 0.127  | 1.025 ± 0.054  | 0.968 ± 0.137  
# ensemble ml   | 0.402          | 0.893          | 0.797          

# ZAP9
# Total models: 10, mean number of epochs: 49.4 ± 13.7
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.529 ± 0.129  | 1.061 ± 0.056  | 0.988 ± 0.056  
# ensemble ml   | 0.428          | 0.964          | 0.875          

# mbondi
# Total models: 10, mean number of epochs: 37.1 ± 8.3
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.709 ± 0.077  | 1.281 ± 0.049  | 1.004 ± 0.076  
# ensemble ml   | 0.609          | 1.184          | 0.897          

# AASC
# Total models: 10, mean number of epochs: 40.0 ± 12.8
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.860 ± 0.154  | 1.346 ± 0.061  | 1.095 ± 0.152  
# ensemble ml   | 0.754          | 1.241          | 0.979          

# IGB5
# Total models: 10, mean number of epochs: 53.6 ± 10.7
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.797 ± 0.143  | 1.370 ± 0.075  | 1.197 ± 0.097  
# ensemble ml   | 0.672          | 1.249          | 1.051          

# ML alone
# Total models: 10, mean number of epochs: 48.1 ± 8.4
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.977 ± 0.256  | 1.468 ± 0.171  | 1.332 ± 0.165  
# ensemble ml   | 0.815          | 1.224          | 1.173          

# delete_me/feats-node_z-hy-ar-de-nh-edge_bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 53.6 ± 22.9
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.523 ± 0.172  | 0.833 ± 0.043  | 0.877 ± 0.082  
# ensemble ml   | 0.424          | 0.707          | 0.766          

# CHA-GB
# Total models: 10, mean number of epochs: 46.0 ± 11.6
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.602 ± 0.184  | 1.049 ± 0.102  | 0.892 ± 0.070  
# ensemble ml   | 0.474          | 0.912          | 0.740          

# ZAP9
# Total models: 10, mean number of epochs: 60.0 ± 18.6
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.512 ± 0.183  | 1.016 ± 0.089  | 0.951 ± 0.116  
# ensemble ml   | 0.407          | 0.910          | 0.823          

# mbondi
# Total models: 10, mean number of epochs: 41.8 ± 10.6
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.854 ± 0.115  | 1.314 ± 0.087  | 1.149 ± 0.057  
# ensemble ml   | 0.746          | 1.203          | 1.060          

# AASC
# Total models: 10, mean number of epochs: 49.1 ± 18.1
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.845 ± 0.206  | 1.298 ± 0.077  | 1.166 ± 0.108  
# ensemble ml   | 0.717          | 1.157          | 1.022          

# IGB5
# Total models: 10, mean number of epochs: 71.3 ± 24.1
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.696 ± 0.173  | 1.302 ± 0.059  | 1.197 ± 0.110  
# ensemble ml   | 0.582          | 1.163          | 1.057          

# ML alone
# Total models: 10, mean number of epochs: 69.5 ± 15.7
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.729 ± 0.152  | 1.406 ± 0.208  | 1.290 ± 0.199  
# ensemble ml   | 0.566          | 1.198          | 1.113          

# delete_me/feats-node_c-z-hy-ar-de-nh-edge_r-bt-sr-co-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 50.4 ± 15.1
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.520 ± 0.150  | 0.792 ± 0.086  | 0.836 ± 0.087  
# ensemble ml   | 0.436          | 0.692          | 0.709          

# CHA-GB
# Total models: 10, mean number of epochs: 50.0 ± 12.1
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.595 ± 0.082  | 1.066 ± 0.032  | 0.945 ± 0.088  
# ensemble ml   | 0.478          | 0.940          | 0.780          

# ZAP9
# Total models: 10, mean number of epochs: 50.7 ± 8.6
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.452 ± 0.119  | 1.000 ± 0.051  | 0.909 ± 0.066  
# ensemble ml   | 0.355          | 0.926          | 0.806          

# mbondi
# Total models: 10, mean number of epochs: 34.6 ± 9.0
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.817 ± 0.222  | 1.281 ± 0.075  | 1.056 ± 0.153  
# ensemble ml   | 0.693          | 1.175          | 0.937          

# AASC
# Total models: 10, mean number of epochs: 31.0 ± 7.6
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.942 ± 0.174  | 1.395 ± 0.057  | 1.207 ± 0.142  
# ensemble ml   | 0.840          | 1.301          | 1.100          

# IGB5
# Total models: 10, mean number of epochs: 57.8 ± 18.9
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.764 ± 0.145  | 1.274 ± 0.056  | 1.144 ± 0.133  
# ensemble ml   | 0.637          | 1.123          | 0.974          

# ML alone
# Total models: 10, mean number of epochs: 67.6 ± 13.0
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.714 ± 0.148  | 1.518 ± 0.221  | 1.318 ± 0.113  
# ensemble ml   | 0.581          | 1.276          | 1.151          

# delete_me/feats-node_c-z-hy-ar-de-nh-edge_r-bt-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 10, mean number of epochs: 59.5 ± 25.0
#               | train          | valid          | test           
# physics model | 1.430          | 1.537          | 1.350          
# physics + ml  | 0.460 ± 0.174  | 0.796 ± 0.073  | 0.885 ± 0.083  
# ensemble ml   | 0.364          | 0.670          | 0.783          

# CHA-GB
# Total models: 10, mean number of epochs: 58.6 ± 19.3
#               | train          | valid          | test           
# physics model | 1.581          | 1.836          | 1.327          
# physics + ml  | 0.566 ± 0.155  | 1.091 ± 0.080  | 0.934 ± 0.061  
# ensemble ml   | 0.432          | 0.955          | 0.787          

# ZAP9
# Total models: 10, mean number of epochs: 54.6 ± 19.6
#               | train          | valid          | test           
# physics model | 1.485          | 1.834          | 1.705          
# physics + ml  | 0.517 ± 0.163  | 0.969 ± 0.084  | 0.901 ± 0.080  
# ensemble ml   | 0.406          | 0.854          | 0.790          

# mbondi
# Total models: 10, mean number of epochs: 38.9 ± 14.7
#               | train          | valid          | test           
# physics model | 2.108          | 2.290          | 2.194          
# physics + ml  | 0.843 ± 0.188  | 1.306 ± 0.067  | 1.124 ± 0.104  
# ensemble ml   | 0.730          | 1.187          | 1.007          

# AASC
# Total models: 10, mean number of epochs: 41.9 ± 10.5
#               | train          | valid          | test           
# physics model | 2.231          | 2.516          | 2.477          
# physics + ml  | 0.860 ± 0.132  | 1.281 ± 0.082  | 1.129 ± 0.097  
# ensemble ml   | 0.777          | 1.162          | 1.027          

# IGB5
# Total models: 10, mean number of epochs: 66.6 ± 15.3
#               | train          | valid          | test           
# physics model | 2.606          | 2.954          | 3.033          
# physics + ml  | 0.745 ± 0.153  | 1.247 ± 0.063  | 1.246 ± 0.144  
# ensemble ml   | 0.633          | 1.090          | 1.071          

# ML alone
# Total models: 10, mean number of epochs: 60.6 ± 6.9
#               | train          | valid          | test           
# physics model | 5.280          | 5.559          | 5.154          
# physics + ml  | 0.787 ± 0.194  | 1.360 ± 0.201  | 1.189 ± 0.167  
# ensemble ml   | 0.629          | 1.144          | 1.030          

# CPU times: user 2h 37min 34s, sys: 2min 8s, total: 2h 39min 42s
# Wall time: 1h 3min 45s

In [ ]:
#geometric features, paper split, filter
# delete_me/feats-node_c-br-edge_r-edges-var_bonds-implicit_h_True/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 2, mean number of epochs: 67.0 ± 11.0
#               | train          | valid          | test           | amino          
# physics model | 1.430          | 1.537          | 1.350          | 0.000          
# physics + ml  | 0.637 ± 0.025  | 0.791 ± 0.006  | 0.980 ± 0.019  | 0.524 ± 0.252  
# ensemble ml   | 0.596          | 0.748          | 0.956          | 0.369          

# CHA-GB
# Total models: 2, mean number of epochs: 60.5 ± 4.5
#               | train          | valid          | test           | amino          
# physics model | 1.581          | 1.836          | 1.327          | 0.864          
# physics + ml  | 0.595 ± 0.061  | 0.999 ± 0.100  | 0.886 ± 0.003  | 3.230 ± 1.963  
# ensemble ml   | 0.513          | 0.911          | 0.741          | 2.598          

# ZAP9
# Total models: 2, mean number of epochs: 54.5 ± 0.5
#               | train          | valid          | test           | amino          
# physics model | 1.485          | 1.834          | 1.705          | 4.554          
# physics + ml  | 0.629 ± 0.097  | 1.070 ± 0.009  | 0.962 ± 0.079  | 4.653 ± 0.203  
# ensemble ml   | 0.574          | 1.025          | 0.884          | 4.623          

# mbondi
# Total models: 2, mean number of epochs: 46.5 ± 5.5
#               | train          | valid          | test           | amino          
# physics model | 2.108          | 2.290          | 2.194          | 1.283          
# physics + ml  | 0.886 ± 0.171  | 1.334 ± 0.054  | 1.153 ± 0.063  | 1.475 ± 0.239  
# ensemble ml   | 0.830          | 1.290          | 1.091          | 1.393          

# AASC
# Total models: 2, mean number of epochs: 51.0 ± 13.0
#               | train          | valid          | test           | amino          
# physics model | 2.231          | 2.516          | 2.477          | 5.996          
# physics + ml  | 0.912 ± 0.147  | 1.283 ± 0.066  | 1.130 ± 0.027  | 4.097 ± 0.050  
# ensemble ml   | 0.868          | 1.250          | 1.099          | 4.077          

# IGB5
# Total models: 2, mean number of epochs: 64.5 ± 7.5
#               | train          | valid          | test           | amino          
# physics model | 2.606          | 2.954          | 3.033          | 3.117          
# physics + ml  | 0.763 ± 0.073  | 1.218 ± 0.029  | 1.283 ± 0.023  | 2.905 ± 1.460  
# ensemble ml   | 0.679          | 1.139          | 1.195          | 2.202          

# ML alone
# Total models: 2, mean number of epochs: 52.0 ± 17.0
#               | train          | valid          | test           | amino          
# physics model | 5.280          | 5.559          | 5.154          | 36.390          
# physics + ml  | 1.042 ± 0.009  | 1.594 ± 0.014  | 1.536 ± 0.049  | 28.366 ± 0.099  
# ensemble ml   | 0.958          | 1.519          | 1.443          | 28.350          

# delete_me/feats-node_c-br-edge_r-edges-var_fc-implicit_h_False/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# TIP3P
# Total models: 2, mean number of epochs: 77.5 ± 23.5
#               | train          | valid          | test           | amino          
# physics model | 1.430          | 1.537          | 1.350          | 0.000          
# physics + ml  | 1.054 ± 0.028  | 1.074 ± 0.002  | 1.155 ± 0.019  | 1.171 ± 0.032  
# ensemble ml   | 1.038          | 1.058          | 1.140          | 1.153          

# CHA-GB
# Total models: 2, mean number of epochs: 63.5 ± 37.5
#               | train          | valid          | test           | amino          
# physics model | 1.581          | 1.836          | 1.327          | 0.864          
# physics + ml  | 1.309 ± 0.064  | 1.499 ± 0.007  | 1.224 ± 0.032  | 1.546 ± 0.045  
# ensemble ml   | 1.271          | 1.475          | 1.193          | 1.489          

# ZAP9
# Total models: 2, mean number of epochs: 44.5 ± 7.5
#               | train          | valid          | test           | amino          
# physics model | 1.485          | 1.834          | 1.705          | 4.554          
# physics + ml  | 1.203 ± 0.076  | 1.426 ± 0.086  | 1.484 ± 0.112  | 5.614 ± 0.201  
# ensemble ml   | 1.148          | 1.352          | 1.444          | 5.607          

In [ ]:
#geometric features, hilo split, filter

In [ ]:
#filter uncertain paper split
# filter_uncertain_prelim/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 62.0 ± 9.0
#               | train          | valid          | test           | amino          
# physics model | 5.280          | 5.559          | 5.154          | 36.390          
# physics + ml  | 0.655 ± 0.001  | 1.334 ± 0.222  | 1.098 ± 0.002  | 29.266 ± 0.328  
# ensemble ml   | 0.576          | 1.189          | 1.033          | 29.260          

# ZAP9
# Total models: 2, mean number of epochs: 45.5 ± 8.5
#               | train          | valid          | test           | amino          
# physics model | 1.485          | 1.834          | 1.705          | 4.554          
# physics + ml  | 0.594 ± 0.062  | 0.999 ± 0.056  | 0.861 ± 0.042  | 3.704 ± 0.101  
# ensemble ml   | 0.532          | 0.935          | 0.809          | 3.698          

# mbondi
# Total models: 2, mean number of epochs: 31.5 ± 3.5
#               | train          | valid          | test           | amino          
# physics model | 2.108          | 2.290          | 2.194          | 1.283          
# physics + ml  | 0.850 ± 0.031  | 1.195 ± 0.029  | 0.995 ± 0.017  | 2.544 ± 0.027  
# ensemble ml   | 0.834          | 1.182          | 0.985          | 2.534          

# TIP3P
# Total models: 2, mean number of epochs: 64.0 ± 12.0
#               | train          | valid          | test           | amino          
# physics model | 1.430          | 1.537          | 1.350          | 0.000          
# physics + ml  | 0.406 ± 0.018  | 0.673 ± 0.004  | 0.748 ± 0.007  | 1.231 ± 0.090  
# ensemble ml   | 0.376          | 0.624          | 0.707          | 1.206          

# filter_uncertain_prelim/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 72.0 ± 1.0
#               | train          | valid          | test           | amino          
# physics model | 5.280          | 5.559          | 5.154          | 36.390          
# physics + ml  | 0.645 ± 0.053  | 1.094 ± 0.030  | 1.043 ± 0.075  | 29.008 ± 0.123  
# ensemble ml   | 0.614          | 1.076          | 0.999          | 29.001          

# ZAP9
# Total models: 2, mean number of epochs: 67.0 ± 34.0
#               | train          | valid          | test           | amino          
# physics model | 1.485          | 1.834          | 1.705          | 4.554          
# physics + ml  | 0.673 ± 0.097  | 0.936 ± 0.030  | 0.936 ± 0.006  | 4.764 ± 0.036  
# ensemble ml   | 0.621          | 0.887          | 0.883          | 4.724          

# mbondi
# Total models: 2, mean number of epochs: 42.5 ± 19.5
#               | train          | valid          | test           | amino          
# physics model | 2.108          | 2.290          | 2.194          | 1.283          
# physics + ml  | 0.895 ± 0.138  | 1.105 ± 0.090  | 0.965 ± 0.039  | 2.490 ± 0.672  
# ensemble ml   | 0.825          | 1.045          | 0.886          | 2.335          

# TIP3P
# Total models: 2, mean number of epochs: 55.0 ± 9.0
#               | train          | valid          | test           | amino          
# physics model | 1.430          | 1.537          | 1.350          | 0.000          
# physics + ml  | 0.503 ± 0.000  | 0.590 ± 0.036  | 1.009 ± 0.060  | 1.250 ± 0.438  
# ensemble ml   | 0.472          | 0.553          | 0.973          | 1.113          

# CPU times: user 13min 59s, sys: 9.25 s, total: 14min 8s
# Wall time: 8min 5s

In [ ]:
#filter uncertain hilo
# filter_uncertain_prelim_hilo/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 101.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 3.779          | 3.861          | 11.655          | 36.390          
# physics + ml  | 0.453 ± 0.075  | 0.814 ± 0.048  | 6.126 ± 0.209  | 31.618 ± 0.113  
# ensemble ml   | 0.416          | 0.768          | 6.111          | 31.618          

# ZAP9
# Total models: 2, mean number of epochs: 36.0 ± 5.0
#               | train          | valid          | test           | amino          
# physics model | 1.366          | 1.389          | 2.621          | 4.554          
# physics + ml  | 0.568 ± 0.090  | 0.612 ± 0.063  | 2.502 ± 0.433  | 3.949 ± 0.467  
# ensemble ml   | 0.528          | 0.576          | 2.412          | 3.898          

# mbondi
# Total models: 2, mean number of epochs: 39.0 ± 4.0
#               | train          | valid          | test           | amino          
# physics model | 1.873          | 1.939          | 3.579          | 1.283          
# physics + ml  | 0.504 ± 0.044  | 0.679 ± 0.040  | 2.063 ± 0.014  | 3.811 ± 0.680  
# ensemble ml   | 0.466          | 0.649          | 2.030          | 3.606          

# TIP3P
# Total models: 2, mean number of epochs: 55.5 ± 11.5
#               | train          | valid          | test           | amino          
# physics model | 1.256          | 1.239          | 2.423          | 0.000          
# physics + ml  | 0.432 ± 0.043  | 0.518 ± 0.002  | 1.901 ± 0.004  | 1.120 ± 0.151  
# ensemble ml   | 0.408          | 0.490          | 1.862          | 1.110          

# filter_uncertain_prelim_hilo/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 48.0 ± 1.0
#               | train          | valid          | test           | amino          
# physics model | 3.779          | 3.861          | 11.655          | 36.390          
# physics + ml  | 0.418 ± 0.031  | 0.642 ± 0.023  | 5.890 ± 0.083  | 31.229 ± 0.084  
# ensemble ml   | 0.390          | 0.614          | 5.887          | 31.228          

# ZAP9
# Total models: 2, mean number of epochs: 45.0 ± 14.0
#               | train          | valid          | test           | amino          
# physics model | 1.366          | 1.389          | 2.621          | 4.554          
# physics + ml  | 0.510 ± 0.132  | 0.662 ± 0.032  | 3.489 ± 0.059  | 6.526 ± 0.044  
# ensemble ml   | 0.476          | 0.632          | 3.468          | 6.523          

# mbondi
# Total models: 2, mean number of epochs: 33.5 ± 8.5
#               | train          | valid          | test           | amino          
# physics model | 1.873          | 1.939          | 3.579          | 1.283          
# physics + ml  | 0.736 ± 0.253  | 0.805 ± 0.031  | 2.979 ± 0.500  | 3.380 ± 1.178  
# ensemble ml   | 0.656          | 0.716          | 2.685          | 3.334          

# TIP3P
# Total models: 2, mean number of epochs: 47.0 ± 9.0
#               | train          | valid          | test           | amino          
# physics model | 1.256          | 1.239          | 2.423          | 0.000          
# physics + ml  | 0.436 ± 0.012  | 0.522 ± 0.017  | 2.624 ± 0.010  | 2.082 ± 0.451  
# ensemble ml   | 0.419          | 0.508          | 2.607          | 2.063          

# CPU times: user 12min 35s, sys: 8.88 s, total: 12min 44s
# Wall time: 7min 7s

In [ ]:
#below are epochs 100 noisy targets prelim

In [ ]:
#with 0.5*uncertainty, paper split
# noisy_targets_prelim/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 1, mean number of epochs: 61.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 5.383          | 5.588          | 5.450          | 36.390          
# physics + ml  | 0.840 ± 0.000  | 1.164 ± 0.000  | 1.522 ± 0.000  | 30.594 ± 0.000  
# ensemble ml   | 0.840          | 1.164          | 1.522          | 30.594          

# ZAP9
# Total models: 1, mean number of epochs: 61.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.607          | 1.847          | 1.855          | 4.554          
# physics + ml  | 0.535 ± 0.000  | 1.054 ± 0.000  | 1.105 ± 0.000  | 5.068 ± 0.000  
# ensemble ml   | 0.535          | 1.054          | 1.105          | 5.068          

# mbondi
# Total models: 1, mean number of epochs: 30.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 2.235          | 2.306          | 2.299          | 1.283          
# physics + ml  | 0.940 ± 0.000  | 1.270 ± 0.000  | 1.201 ± 0.000  | 3.250 ± 0.000  
# ensemble ml   | 0.940          | 1.270          | 1.201          | 3.250          

# TIP3P
# Total models: 1, mean number of epochs: 47.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.559          | 1.567          | 1.435          | 0.000          
# physics + ml  | 0.647 ± 0.000  | 0.679 ± 0.000  | 0.779 ± 0.000  | 1.450 ± 0.000  
# ensemble ml   | 0.647          | 0.679          | 0.779          | 1.450          

# noisy_targets_prelim/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 1, mean number of epochs: 40.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 5.383          | 5.588          | 5.450          | 36.390          
# physics + ml  | 1.092 ± 0.000  | 1.187 ± 0.000  | 1.411 ± 0.000  | 29.265 ± 0.000  
# ensemble ml   | 1.092          | 1.187          | 1.411          | 29.265          

# ZAP9
# Total models: 1, mean number of epochs: 67.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.607          | 1.847          | 1.855          | 4.554          
# physics + ml  | 0.648 ± 0.000  | 0.939 ± 0.000  | 1.168 ± 0.000  | 4.574 ± 0.000  
# ensemble ml   | 0.648          | 0.939          | 1.168          | 4.574          

# mbondi
# Total models: 1, mean number of epochs: 18.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 2.235          | 2.306          | 2.299          | 1.283          
# physics + ml  | 1.146 ± 0.000  | 1.212 ± 0.000  | 1.122 ± 0.000  | 3.204 ± 0.000  
# ensemble ml   | 1.146          | 1.212          | 1.122          | 3.204          

# TIP3P
# Total models: 1, mean number of epochs: 57.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.559          | 1.567          | 1.435          | 0.000          
# physics + ml  | 0.651 ± 0.000  | 0.653 ± 0.000  | 1.055 ± 0.000  | 1.265 ± 0.000  
# ensemble ml   | 0.651          | 0.653          | 1.055          | 1.265          

# CPU times: user 7min 4s, sys: 4.48 s, total: 7min 9s
# Wall time: 4min 20s

In [ ]:
#with 0.5*uncertainty, hilo_split
# noisy_targets_prelim_hilo/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 1, mean number of epochs: 61.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 3.826          | 3.852          | 11.486          | 36.390          
# physics + ml  | 0.538 ± 0.000  | 0.719 ± 0.000  | 5.826 ± 0.000  | 31.554 ± 0.000  
# ensemble ml   | 0.538          | 0.719          | 5.826          | 31.554          

# ZAP9
# Total models: 1, mean number of epochs: 42.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.500          | 1.439          | 2.601          | 4.554          
# physics + ml  | 0.566 ± 0.000  | 0.621 ± 0.000  | 2.952 ± 0.000  | 5.126 ± 0.000  
# ensemble ml   | 0.566          | 0.621          | 2.952          | 5.126          

# mbondi
# Total models: 1, mean number of epochs: 36.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 2.030          | 2.015          | 3.435          | 1.283          
# physics + ml  | 0.608 ± 0.000  | 0.702 ± 0.000  | 2.115 ± 0.000  | 2.653 ± 0.000  
# ensemble ml   | 0.608          | 0.702          | 2.115          | 2.653          

# TIP3P
# Total models: 1, mean number of epochs: 35.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.372          | 1.301          | 2.470          | 0.000          
# physics + ml  | 0.610 ± 0.000  | 0.507 ± 0.000  | 2.130 ± 0.000  | 0.725 ± 0.000  
# ensemble ml   | 0.610          | 0.507          | 2.130          | 0.725          

# noisy_targets_prelim_hilo/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_100-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 1, mean number of epochs: 61.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 3.826          | 3.852          | 11.486          | 36.390          
# physics + ml  | 0.423 ± 0.000  | 0.619 ± 0.000  | 5.552 ± 0.000  | 31.343 ± 0.000  
# ensemble ml   | 0.423          | 0.619          | 5.552          | 31.343          

# ZAP9
# Total models: 1, mean number of epochs: 59.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.500          | 1.439          | 2.601          | 4.554          
# physics + ml  | 0.633 ± 0.000  | 0.602 ± 0.000  | 3.674 ± 0.000  | 6.841 ± 0.000  
# ensemble ml   | 0.633          | 0.602          | 3.674          | 6.841          

# mbondi
# Total models: 1, mean number of epochs: 32.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 2.030          | 2.015          | 3.435          | 1.283          
# physics + ml  | 0.719 ± 0.000  | 0.755 ± 0.000  | 2.545 ± 0.000  | 4.854 ± 0.000  
# ensemble ml   | 0.719          | 0.755          | 2.545          | 4.854          

# TIP3P
# Total models: 1, mean number of epochs: 41.0 ± 0.0
#               | train          | valid          | test           | amino          
# physics model | 1.372          | 1.301          | 2.470          | 0.000          
# physics + ml  | 0.617 ± 0.000  | 0.569 ± 0.000  | 2.444 ± 0.000  | 2.519 ± 0.000  
# ensemble ml   | 0.617          | 0.569          | 2.444          | 2.519          

# CPU times: user 6min 56s, sys: 4.56 s, total: 7min 1s
# Wall time: 4min 16s

In [ ]:
#below are epochs 101

In [ ]:
#with 1.0, paper split
# noisy_targets_prelim/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 60.5 ± 3.5
#               | train          | valid          | test           | amino          
# physics model | 5.383          | 5.588          | 5.450          | 36.390          
# physics + ml  | 0.813 ± 0.050  | 1.370 ± 0.114  | 1.411 ± 0.072  | 29.511 ± 0.408  
# ensemble ml   | 0.732          | 1.291          | 1.364          | 29.505          

# ZAP9
# Total models: 2, mean number of epochs: 53.5 ± 24.5
#               | train          | valid          | test           | amino          
# physics model | 1.607          | 1.847          | 1.855          | 4.554          
# physics + ml  | 0.729 ± 0.094  | 0.997 ± 0.085  | 1.220 ± 0.095  | 4.066 ± 0.562  
# ensemble ml   | 0.661          | 0.959          | 1.168          | 4.006          

# mbondi
# Total models: 2, mean number of epochs: 31.0 ± 1.0
#               | train          | valid          | test           | amino          
# physics model | 2.235          | 2.306          | 2.299          | 1.283          
# physics + ml  | 0.974 ± 0.005  | 1.229 ± 0.006  | 1.153 ± 0.082  | 3.006 ± 0.673  
# ensemble ml   | 0.932          | 1.182          | 1.119          | 2.868          

# TIP3P
# Total models: 2, mean number of epochs: 40.5 ± 2.5
#               | train          | valid          | test           | amino          
# physics model | 1.559          | 1.567          | 1.435          | 0.000          
# physics + ml  | 0.830 ± 0.038  | 0.812 ± 0.046  | 0.943 ± 0.043  | 1.325 ± 0.083  
# ensemble ml   | 0.799          | 0.781          | 0.905          | 1.299          

# noisy_targets_prelim/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 48.0 ± 14.0
#               | train          | valid          | test           | amino          
# physics model | 5.383          | 5.588          | 5.450          | 36.390          
# physics + ml  | 0.953 ± 0.161  | 1.230 ± 0.049  | 1.384 ± 0.069  | 28.688 ± 0.194  
# ensemble ml   | 0.880          | 1.136          | 1.267          | 28.671          

# ZAP9
# Total models: 2, mean number of epochs: 68.5 ± 7.5
#               | train          | valid          | test           | amino          
# physics model | 1.607          | 1.847          | 1.855          | 4.554          
# physics + ml  | 0.782 ± 0.024  | 0.948 ± 0.029  | 1.151 ± 0.048  | 4.996 ± 0.145  
# ensemble ml   | 0.734          | 0.909          | 1.108          | 4.981          

# mbondi
# Total models: 2, mean number of epochs: 50.0 ± 1.0
#               | train          | valid          | test           | amino          
# physics model | 2.235          | 2.306          | 2.299          | 1.283          
# physics + ml  | 0.911 ± 0.001  | 1.155 ± 0.007  | 1.085 ± 0.045  | 2.579 ± 0.039  
# ensemble ml   | 0.863          | 1.121          | 1.048          | 2.548          

# TIP3P
# Total models: 2, mean number of epochs: 85.0 ± 16.0
#               | train          | valid          | test           | amino          
# physics model | 1.559          | 1.567          | 1.435          | 0.000          
# physics + ml  | 0.672 ± 0.097  | 0.649 ± 0.010  | 1.010 ± 0.026  | 1.222 ± 0.152  
# ensemble ml   | 0.627          | 0.565          | 0.975          | 1.152          

# CPU times: user 14min 43s, sys: 9.73 s, total: 14min 53s
# Wall time: 8min 46s

In [ ]:
#with 1.0, hilo_split
# noisy_targets_prelim_hilo/feats-node_z-ic-hy-hb-ar-de-nh-edge_bt-sr-co-sc-edges-var_bonds-implicit_h_True/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 63.0 ± 2.0
#               | train          | valid          | test           | amino          
# physics model | 3.826          | 3.852          | 11.486          | 36.390          
# physics + ml  | 0.546 ± 0.007  | 0.756 ± 0.029  | 5.739 ± 0.044  | 31.532 ± 0.070  
# ensemble ml   | 0.511          | 0.729          | 5.735          | 31.532          

# ZAP9
# Total models: 2, mean number of epochs: 89.5 ± 11.5
#               | train          | valid          | test           | amino          
# physics model | 1.500          | 1.439          | 2.601          | 4.554          
# physics + ml  | 0.476 ± 0.028  | 0.607 ± 0.080  | 2.690 ± 0.031  | 3.700 ± 0.061  
# ensemble ml   | 0.401          | 0.561          | 2.644          | 3.683          

# mbondi
# Total models: 2, mean number of epochs: 90.0 ± 11.0
#               | train          | valid          | test           | amino          
# physics model | 2.030          | 2.015          | 3.435          | 1.283          
# physics + ml  | 0.549 ± 0.001  | 0.770 ± 0.062  | 2.200 ± 0.004  | 3.374 ± 0.077  
# ensemble ml   | 0.504          | 0.738          | 2.095          | 3.296          

# TIP3P
# Total models: 2, mean number of epochs: 39.5 ± 8.5
#               | train          | valid          | test           | amino          
# physics model | 1.372          | 1.301          | 2.470          | 0.000          
# physics + ml  | 0.658 ± 0.021  | 0.566 ± 0.020  | 2.124 ± 0.120  | 1.018 ± 0.074  
# ensemble ml   | 0.627          | 0.527          | 2.097          | 0.980          

# noisy_targets_prelim_hilo/feats-node_c-gb-sgb-br-z-ic-hy-hb-ar-de-nh-edge_r-gb-bt-sr-co-sc-edges-var_fc-implicit_h_False/ep_101-over_1_0-opat_20-del_0_01-cpat_20/
# ML alone
# Total models: 2, mean number of epochs: 69.0 ± 32.0
#               | train          | valid          | test           | amino          
# physics model | 3.826          | 3.852          | 11.486          | 36.390          
# physics + ml  | 0.531 ± 0.069  | 0.694 ± 0.003  | 5.706 ± 0.108  | 31.316 ± 0.041  
# ensemble ml   | 0.469          | 0.647          | 5.699          | 31.314          

# ZAP9
# Total models: 2, mean number of epochs: 53.5 ± 9.5
#               | train          | valid          | test           | amino          
# physics model | 1.500          | 1.439          | 2.601          | 4.554          
# physics + ml  | 0.643 ± 0.124  | 0.666 ± 0.013  | 3.297 ± 0.246  | 6.398 ± 0.088  
# ensemble ml   | 0.592          | 0.610          | 3.263          | 6.389          

# mbondi
# Total models: 2, mean number of epochs: 42.0 ± 10.0
#               | train          | valid          | test           | amino          
# physics model | 2.030          | 2.015          | 3.435          | 1.283          
# physics + ml  | 0.687 ± 0.093  | 0.744 ± 0.018  | 2.885 ± 0.045  | 4.482 ± 0.486  
# ensemble ml   | 0.650          | 0.720          | 2.805          | 4.447          

# TIP3P
# Total models: 2, mean number of epochs: 49.0 ± 25.0
#               | train          | valid          | test           | amino          
# physics model | 1.372          | 1.301          | 2.470          | 0.000          
# physics + ml  | 0.672 ± 0.072  | 0.575 ± 0.016  | 2.330 ± 0.065  | 1.881 ± 0.035  
# ensemble ml   | 0.649          | 0.541          | 2.305          | 1.865          

# CPU times: user 14min 48s, sys: 9.72 s, total: 14min 58s
# Wall time: 8min 25s